<a href="https://colab.research.google.com/github/JorgeTonatiuh/Tarea_1/blob/main/retropropagacion_en_red_densa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retropropagación en red densa.

Programa el algoritmo de retropropagación usando NumPy para una tarea de clasificación binaria presuponiendo una red densa con dos capas ocultas. Esta red tiene una función de activación logística en todas sus neuronas y se entrena minimizando la función de pérdida de entropía cruzada binaria. Describe las fórmulas y reglas de actualización de los pesos y sesgos de cada capa y entrena y evalúa la red en algún conjunto de datos.

In [35]:
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Dataset

Para entrenar el modelo se utilizará el dataset de flores iris el cual contiene la información de 3 clases de estas flores.

1.- Las 3 clases de iris son: 

1.   Iris Setosa
2.   Iris Versicolour
3.   Iris Virginica

2.- Los atributos que contiene el dataset son:

1.   Longitud del sépalo
2.   Ancho del sépalo
3.   Longitud del pétalo
4.   Ancho del pétalo

3.- Número de instancias: 150 (50 en cada una de las tres clases)

4.- Este dataset no contiene valores faltantes

In [21]:
#Se obiene el dataset utilizando sklearn datasets
iris = datasets.load_iris()

In [20]:
#Se obtiene la data de los atributos
data = iris.data

In [33]:
#Se obtiene el target
#Dado que las clases estan representadas numericamente en el dataset original
#se transforman estas represaentaciones aplicando One-hot encoding
s = pd.Series(iris.target)
target = pd.get_dummies(s).to_numpy()

Dividimos aleatoriamente el conjunto de datos en 80% para entrenamiento y 20% para validación. Para eso usaremos la funcion train_test_split de scikit-learn.

In [36]:
X_ent, X_valid, y_ent, y_valid = train_test_split(data, target, test_size=0.2, random_state=42)

## Entrenamiento del modelo

Se calcularan los pesos para una red densa con dos capas ocultas.

In [37]:
# Inicializamos los pesos con números aleatorios
w = np.random.rand(X_ent.shape[1], 1)
# Inicializamos el sesgo con 0
b = np.zeros((1, 1))

In [40]:
X_ent.shape[0]

120